# Playing with MPS
Start by copying the part of the code that transforms a generic state to an MPS from the sixth class

In [1]:
import numpy as np 
import scipy.linalg as LA
def generate_random_state(N):
        dim_h =2**N
        init_state = np.zeros([dim_h,1])
        init_state[0]=1.
        random_h = np.array(np.random.rand(dim_h,dim_h)+1j*np.random.rand(dim_h,dim_h))
        random_h = random_h+random_h.T.conj()
        random_h = random_h/LA.norm(random_h)*N

        random_unitary =LA.expm(-1j*random_h)
        random_state=random_unitary@init_state
        return random_state
    
def compute_A_sigma_bulk(remaining_state,n,list_sigma):
        remaining_spins= len(remaining_state.shape)
        initial_shape =remaining_state.shape
        if n == 0 :
            #print('first')
            first_size =remaining_state.shape[0]
            second_size=np.prod(remaining_state.shape[1:])
        else:
            #print('bulk')
            first_size =remaining_state.shape[0]
            second_size=np.prod(remaining_state.shape[1:])
            remaining_state_c =remaining_state.reshape(first_size,second_size)
            first_size =np.prod(remaining_state.shape[0:2])
            second_size=np.prod(remaining_state.shape[2:])
            sigma_prev = list_sigma[n-1]
            remaining_state = sigma_prev@remaining_state_c
            remaining_state = remaining_state.reshape(initial_shape)
            
            
            
        remaining_state_matrix = remaining_state.reshape(
            first_size,second_size)
        
        A,sigma,rest = LA.svd(remaining_state_matrix,full_matrices
                             =False)
        
        chi=rest.shape[0]
        sigma =np.diag(sigma)
        if n == 0:
            rest_tensor =rest.reshape([rest.shape[0]]
                                  +list(remaining_state.shape[1:]))
        else:
            A =A.reshape(remaining_state.shape[0],remaining_state.shape[1],
                     A.shape[1])
            A = np.einsum('ij,jkl->ikl',LA.pinv(list_sigma[n-1]),A)
            rest_tensor =rest.reshape([rest.shape[0]]+
                                  list(remaining_state.shape[2:]))
        
        return A, sigma, rest_tensor
        
def state_reconstruct_einsum(list_A,list_sigma):
    N =len(list_A)
    avail_index = 'abcdefghlmnopqrstuvz'
    first_piece = list_A_tensors[0]
    for n in range(N-1):
        if n ==0:
            first_piece=np.einsum('ab,bd->ad',first_piece,list_sigma_tensors[n])
        else: 
            #print(first_piece.shape)
            #print(list_sigma_tensors[n].shape)
            num_left_indices=len(first_piece.shape)
            einsum_index=avail_index[0:num_left_indices]
            einsum_index+=','+avail_index[num_left_indices-1:num_left_indices+1]
            einsum_index+='->'+avail_index[0:num_left_indices-1]+avail_index[
               num_left_indices]
            #print(einsum_index)
            first_piece=np.einsum(einsum_index,
                                      first_piece,list_sigma_tensors[n])
        print('contracting '+str(n))
        num_left_indices=len(first_piece.shape)
        einsum_index=avail_index[0:num_left_indices]
        next_piece = list_A_tensors[n+1]
        other_index=len(next_piece.shape)
        einsum_index = einsum_index+','+avail_index[num_left_indices-1:
                    num_left_indices+other_index-1]
        einsum_index += '->'+avail_index[0:num_left_indices-1]+ avail_index[
            num_left_indices:num_left_indices+other_index-1]
        #print(einsum_index)
        first_piece =np.einsum(einsum_index,first_piece,next_piece)    
    return first_piece
def state_reconstruct_matmult(list_A,list_sigma):
    N =len(list_A)
    #print(N)
    list_d =[]
    list_d.append(list_A[0].shape[1])
    mat_left= list_A[0]
    for n in range(1,N-1):
        #print(n)
        mat_left = mat_left@list_sigma[n-1]
        mat_left = mat_left@list_A[n].reshape(list_A[n].shape[0],list_A[n].shape[1]*list_A[n].shape[2])
        mat_left = mat_left.reshape(mat_left.shape[0]*list_A[n].shape[1],list_A[n].shape[2])
        list_d.append(list_A[n].shape[1])
    
    mat_left = mat_left@list_sigma[N-2]
    mat_left = mat_left@list_A[N-1]
    list_d.append(list_A[N-1].shape[1])
    mat_left = mat_left.reshape(list_d)
    return mat_left


def create_MPS(N,state):
    remaining_state =state
    list_A_tensors =[]
    list_sigma_tensors =[]
    for n in range(N-1):
    #print(n)
        A,sigma,rest=compute_A_sigma_bulk(remaining_state,n,list_sigma_tensors)
    
        remaining_state =rest
    #print(rest.shape)
        list_A_tensors.append(A)
        list_sigma_tensors.append(sigma)

    
    list_A_tensors.append(rest)
    return list_A_tensors, list_sigma_tensors

Now let start generating some interesting states and obtain their MPS decomposition. Let's first generate the $W$ state:  
$$ |W  \rangle =\frac{1}{\sqrt{N}}\sum_{n=1}^{N} |0 \cdots 0 1_n 0\cdots 0\rangle  $$

In [2]:
N=10
d=2

w_state = np.zeros(d**N)
dimensions =[d]*N
w_state = w_state.reshape(dimensions)
for k in range(N):
    vector =[int(0)]*N
    vector[k]=int(1)
    #print(vector)
    w_state[tuple(vector)]=1.       


#for k in range(d): 
#    for j in range(d):
#         for l in range(d):
#                for m in range(d):
#                    print('[' + str(l)+','+str(j)+','+str(k)+','+str(m)+']')
#                    print(w_state[m,l,j,k])

w_state = w_state/np.sqrt(N)
print(LA.norm(w_state.reshape(d**N)))


1.0


Let' s transform it into an MPS using the function defined abvove

In [ ]:
list_A_tensors,list_sigma_tensors =create_MPS(N,w_state)
recontructed_state_mat_mult = state_reconstruct_matmult(list_A_tensors,list_sigma_tensors)
recontructed_state_mat_mult
np.max(w_state-recontructed_state_mat_mult)

Let's have a look to the singular value of the $|W \rangle $ state

In [ ]:
cut_off =1.e-12
for n in range(N-1):
    eig_rdm =np.diag(list_sigma_tensors[n])
    #print(eig_rdm)
    mask_sl=np.greater_equal(eig_rdm,cut_off)
    non_zero_eig= eig_rdm[mask_sl]
    print(non_zero_eig)
    print(np.sum(non_zero_eig**2))
    

Now let's build the MPS for the state $WW$
$$ |W W \rangle =\frac{1}{\sqrt{N-1}}\sum_{n=1}^{N-1} |0 \cdots 0 1_n 1_{n+1} 0\cdots 0\rangle  $$

In [3]:
ww_state = np.zeros(d**N)
dimensions =[d]*N
ww_state = ww_state.reshape(dimensions)
for k in range(N-1):
    vector =[int(0)]*N
    vector[k]=int(1)
    vector[k+1]=int(1)
    #print(vector)
    ww_state[tuple(vector)]=1.       


#for k in range(d): 
#    for j in range(d):
#         for l in range(d):
#                for m in range(d):
#                    print('[' + str(l)+','+str(j)+','+str(k)+','+str(m)+']')
#                    print(w_state[m,l,j,k])

ww_state = ww_state/np.sqrt(N-1)
print(LA.norm(ww_state.reshape(d**N)))


0.9999999999999999


In [4]:
list_A_tensors,list_sigma_tensors =create_MPS(N,ww_state)
recontructed_state_mat_mult = state_reconstruct_matmult(list_A_tensors,list_sigma_tensors)
recontructed_state_mat_mult
np.max(ww_state-recontructed_state_mat_mult)

2.7755575615628914e-16

In [6]:
cut_off =1.e-12
for n in range(N-1):
    eig_rdm =np.diag(list_sigma_tensors[n])
    #print(eig_rdm)
    mask_sl=np.greater_equal(eig_rdm,cut_off)
    non_zero_eig= eig_rdm[mask_sl]
    print(non_zero_eig)
    print(np.sum(non_zero_eig**2))
    

[0.94280904 0.33333333]
1.0
[0.8819171  0.33333333 0.33333333]
1.0
[0.81649658 0.47140452 0.33333333]
0.9999999999999998
[0.74535599 0.57735027 0.33333333]
0.9999999999999998
[0.66666667 0.66666667 0.33333333]
0.9999999999999996
[0.74535599 0.57735027 0.33333333]
0.9999999999999996
[0.81649658 0.47140452 0.33333333]
0.9999999999999996
[0.8819171  0.33333333 0.33333333]
0.9999999999999994
[0.94280904 0.33333333]
0.9999999999999996


Now define a function that performs the truncation on the MPS you have obtained

In [8]:
def truncate_mps(list_A,list_s,cut_off):
    new_list_A =[]
    new_list_s =[]
    old_list_A =np.copy(list_A)
    for k in range(len(list_A)-1):
        eig_diag = np.diag(list_s[k])
        mask_sl=np.greater_equal(np.abs(eig_diag),cut_off)
        #print(k)
        print (len(mask_sl))
        if k==0:
            new_list_A.append(old_list_A[k][:,mask_sl])
            old_list_A[k+1]=list_A[k+1][mask_sl,:,:]
        else:
            new_list_A.append(old_list_A[k][:,:,mask_sl])
            if k< N-2:
                old_list_A[k+1] = old_list_A[k+1][mask_sl,:,:]
            else:
                old_list_A[k+1] = old_list_A[k+1][mask_sl,:]

        new_list_s.append(list_s[k][mask_sl,mask_sl])
    
    new_list_A.append(old_list_A[k+1])
    return new_list_A, new_list_s


new_list_A,new_list_sigma =truncate_mps(list_A_tensors,list_sigma_tensors,cut_off)

2
4
8
16
32
16
8
4
2


In [14]:
recontructed_state_mat_mult = state_reconstruct_matmult(new_list_A,new_list_sigma)
recontructed_state_mat_mult
np.max(ww_state-recontructed_state_mat_mult)

ValueError: cannot reshape array of size 6 into shape (12,3)

# Matrix product operators

Now let's define a vector of operators, $v^i_{a,b} = \sigma^i_{a,b}$ where we select only a 2-dimensional subspace of the operator space, say $1, \sigma_x$

In [17]:
operators_vect =np.zeros([4,2,2])
identity = np.eye(2)
sigma_x = np.array([[0.,1.],[1.,0.]])
sigma_y = np.array([[0.,-1.j],[1.j,0.]])
sigma_z = np.array([[1.,0.],[0.,-1.]])
operators_vect[0,:,:]=identity
operators_vect[1,:,:]=sigma_x

Now contract the first leg of the operator tensor with the physical leg of the MPS for the $WW$ state, what do you obtain?